In [2]:
import pandas as pd
import numpy as np
import pycaret as pc
from utils import utils
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
#https://www.pyimagesearch.com/2021/05/06/implementing-feedforward-neural-networks-with-keras-and-tensorflow/




2021-09-19 13:57:28.530931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-19 13:57:28.530946: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# feature_df_9 = pd.read_csv('data/color_refined_molecules_9_hop.csv', index_col=0)
# feature_df_8 = pd.read_csv('data/color_refined_molecules_8_hop.csv', index_col=0)
feature_df_7 = pd.read_csv('data/color_refined_molecules_7_hop.csv', index_col=0)
feature_df_6 = pd.read_csv('data/color_refined_molecules_6_hop.csv', index_col=0)
feature_df_5 = pd.read_csv('data/color_refined_molecules_5_hop.csv', index_col=0)
feature_df_4 = pd.read_csv('data/color_refined_molecules_4_hop.csv', index_col=0)

# I'm just using all the colors and seeing what happens.
target_df = utils.load_graph_repersentations()[['target']]

In [4]:
df = pd.concat([feature_df_7,feature_df_6,feature_df_5,feature_df_4, target_df], axis=1)
df

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,target
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12087,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12088,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# baseline model
import tensorflow as tf
from sklearn.metrics import precision_recall_curve, auc
from tensorflow.keras.optimizers import SGD


def compute_auprc(model, X_test, y_test):
    # https://stats.stackexchange.com/questions/157012/area-under-precision-recall-curve-auc-of-pr-curve-and-average-precision-ap
    precision, recall, thresholds = precision_recall_curve(y_test, model.predict(X_test))
    area = auc(recall, precision)
    return area

def get_optimizer():
	epochs=400
	learning_rate = 0.1
	decay_rate = learning_rate / epochs
	momentum = 0.8
	sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
	return sgd

# https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594
def create_baseline():
	# create model
	model = Sequential()
    # baseline is a single fully connected layer of the same size as the input deminsion
	model.add(Dense(1000, input_dim=20_000, activation='relu'))
	model.add(Dense(500, activation='relu'))
	model.add(Dense(64, activation='relu'))
	model.add(Dense(64, activation='relu'))
	model.add(Dense(64, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model

	#

	model.compile(loss='binary_crossentropy', optimizer=get_optimizer(), metrics=['accuracy'])
	return model

In [6]:
from sklearn.model_selection import train_test_split

scores = []
X, Y = df.iloc[:,:-1].values, df.iloc[:,-1].values

for i in range(10):
    model = create_baseline()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=i)
    model.fit(X_train, y_train, validation_data=(X_test, y_test))
    score = compute_auprc(model, X_test, y_test)
    print(f'AUPRC : {score}')
    scores.append(score)



2021-09-19 13:57:47.543984: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-19 13:57:47.544269: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-19 13:57:47.544418: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-09-19 13:57:47.544458: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-09-19 13:57:47.544496: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Epoch 1/400
303/303 [==============================] - 25s 83ms/step - loss: 0.5318 - accuracy: 0.7328 - val_loss: 0.4757 - val_accuracy: 0.7693
Epoch 2/400
303/303 [==============================] - 24s 79ms/step - loss: 0.2328 - accuracy: 0.9101 - val_loss: 0.5314 - val_accuracy: 0.7582
Epoch 3/400
203/303 [===================>..........] - ETA: 7s - loss: 0.0755 - accuracy: 0.9735

KeyboardInterrupt: 

AttributeError: module 'tensorflow._api.v2.train' has no attribute 'GradientDescentOptimizer'

# evaluate model